## Decorators

Bude nezbytné mít přesně vysvětlený cod. V podstatě dekorátor lze vytvořit pomocí closures - jen se tomu tak neříká.

- decorátor používá funkci jako argument
- a navrací closure
- closure většinou používá jakoukoliv kombinace parametrů 
- díky tomu dosáhneme snadné zavolání
- closure nám většinou dodává extra kod navíc

Základní logika decorátoru.

![Loss](deco1.png)

### Decorators 

Můžeme ale vytvářet decorátory bez použití @ - prostě jen closure. Nemusím poté přiřazovat, že funkce je funkce co volá funkci.

- je třeba dát bacha, že když poté voláme deokorovanou funkci - získáváme informace o decorátoru a ne o funkci kterou dekorujeme - jméno nebo help apod.
- takže například přepíšeme name a doc property v dekorátoru --> ale asi mimo nikdy nebudu vědět jméno funkce
- máme na to functools.wraps - budeme dekorovat dekorátor :)
- V PODSTATĚ DEKORÁTOR MUSÍM OBOHATIT O DALŠÍ DEKORÁTOR pro 100% funčknost

In [2]:
def counter(fn): #funkce co bere funkce jako argument
    count = 0 #základní stav počítadla
    
    def inner(*args, **kwargs): #bude volat fn - nevíme jaká bude proto args a kwargs
        nonlocal count #aby se count zvyšoval v celku ne jenom v inner 
        count +=1
        print(f"Function {fn.__name__} was called {count} times") 
        #vypíšeme zprávu o zavolání funkce
        return fn(*args, **kwargs) #a zavoláme funkci fn s tím co obdrží inner
    
    return inner
#naše counter funkce tedy obdržé funkci - projede jí s inner a navrátí projedou 2 funkce
#kde naše inner funkce obohacuje fn funkci v counter

In [5]:
def add(a: int,b: int =0):
    """adds two values"""
    
    return a + b

In [7]:
help(add) #jasné vše funguje

Help on function add in module __main__:

add(a: int, b: int = 0)
    adds two values



In [9]:
id(add) #bude důležité na potom :)

1995916212832

In [10]:
add = counter(add) #nové add se rovná counter add

In [13]:
id(add) #již se jedná o jinou funkci  - náš closure

1995914399024

In [15]:
help(add) #a též vidíme že máme help na naší inner funkci v closure

Help on function inner in module __main__:

inner(*args, **kwargs)



In [18]:
add(10,20) #provede to funkci add a i funkci inner - skrze náš counter - náš decorator

Function add was called 3 times


30

In [19]:
def mult(a: int, b: int, c:int = 1, *, d):
    """multiplis four values"""
    return a*b*c*d

In [21]:
mult(1,2,3,d=4) #opáčko keyword only argument po *

24

In [23]:
mult(1,2,d=3) #c máme defaultní

6

In [25]:
mult = counter(mult) #help i id je ztraceno

In [26]:
mult(1,2,3,d=4) #ukázka že si náš counter poradí se vším díky args kwargs zápisu

Function inner was called 1 times
Function mult was called 1 times


24

In [28]:
@counter
def my_func(s: str, i: int) -> str:
    return s*i
#my_func = counter(my_func) - stejné jako když použiji decorator

In [30]:
help(my_func) #a jsme zase u inneru :) - decotáro odmaže polohu

Help on function inner in module __main__:

inner(*args, **kwargs)



In [31]:
my_func("a",10)

Function my_func was called 1 times


'aaaaaaaaaa'

In [35]:
from functools import wraps #náš decorátor na dekorátory

In [37]:
def counter(fn): #funkce co bere funkce jako argument
    count = 0 #základní stav počítadla
    
    @wraps(fn) #dekoruj inner pomocí wraps a má původní je fn
    def inner(*args, **kwargs): #bude volat fn - nevíme jaká bude proto args a kwargs
        """
        Naše inner funkce
        """
        nonlocal count #aby se count zvyšoval v celku ne jenom v inner 
        count +=1
        print(f"Function {fn.__name__} was called {count} times") 
        #vypíšeme zprávu o zavolání funkce
        return fn(*args, **kwargs) #a zavoláme funkci fn s tím co obdrží inner
    #též mohu wrapsíka zapsat takto
    #inner = wraps(fn) (inner) #ok divný to záps
    return inner


In [38]:
def mult(a: int, b: int, c:int = 1, *, d):
    """multiplis four values"""
    return a*b*c*d

In [39]:
mult = counter(mult)

In [40]:
help(mult)

Help on function mult in module __main__:

mult(a: int, b: int, c: int = 1, *, d)
    multiplis four values



Tak a takto nám vše funguje.